# Edad de los ladrones de guante blanco

Fuente: Statistics in Criminal Justice, David Weisburd, Chester Britt, https://doi.org/10.1007/978-1-4614-9170-5

**Hipótesis nula**: La edad media de los tres delitos es igual

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

## Datos

Edades por delito

In [ ]:
guante_blanco = pd.read_csv("guante_blanco.csv", index_col=0)
print(guante_blanco)

In [ ]:
sample_size = guante_blanco.size
sample_size

In [ ]:
group_size = len(guante_blanco.index)
group_size

In [ ]:
number_of_groups = len(guante_blanco.columns)
number_of_groups

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_xlabel("Delito")
ax1.set_ylabel("Edad")
ax1.errorbar(x=guante_blanco.columns, y=guante_blanco.mean(), yerr=guante_blanco.std(), ls='', marker='o')

## Estadístico F observado

*Nota*: Las varianzas muestrales usan el denominador n-1 (corrección de Bessel)

In [ ]:
age_mean = guante_blanco.mean()
square_sum_between = group_size * (number_of_groups-1) * age_mean.var()
group_var = guante_blanco.var()
square_sum_within = (group_size-1) * group_var.sum()
ndof_between = number_of_groups - 1
ndof_within = sample_size - number_of_groups
F_observed = ( square_sum_between / ndof_between ) / (square_sum_within / ndof_within)
F_observed

## Pvalor

In [ ]:
F_distribution = scipy.stats.f(ndof_between, ndof_within)
pvalue = F_distribution.sf(F_observed)
pvalue

## Test de Fisher con scipy

In [ ]:
print(*guante_blanco.values.T)

In [ ]:
F_observed, pvalue = scipy.stats.f_oneway(*guante_blanco.values.T)
print(f"F observed = {F_observed}")
print(f"Pvalue = {pvalue}")

## Plot

In [ ]:
fig2, ax2 = plt.subplots()
ax2.set_xlabel("F statistic")
ax2.set_ylabel("PDF")
x = np.linspace(0, 20, 100)
y = scipy.stats.f.pdf(x, ndof_between, ndof_within)
ax2.plot(x,y)
trans = ax2.get_xaxis_transform()
ax2.axvline(F_observed, color='tab:orange', ls='--')
plt.text(F_observed, .5, r" $F_{observed}$", transform=trans)